In [7]:
import numpy as np
# 16个传感器位置
SensorPosition = np.array([
    [[0.12], [0.12], [0]],
    [0.12], [0.04], [0],
    [[0.12], [-0.04], [0]],
    [[0.12], [-0.12], [0]],
    [[0.04], [0.12], [0]],
    [[0.04], [0.04], [0]],
    [[0.04], [-0.04], [0]],
    [[0.04], [-0.12], [0]],
    [[-0.04], [0.12], [0]],
    [[-0.04], [0.04], [0]],
    [[-0.04], [-0.04], [0]],
    [[-0.04], [-0.12], [0]],
    [[-0.12], [0.12], [0]],
    [[-0.12], [0.04], [0]],
    [[-0.12], [-0.04], [0]],
    [[-0.12], [-0.12], [0]]
     ])

<ipython-input-7-9502b000083e>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  SensorPosition = np.array([


In [8]:
# 磁铁位置范围
length=200  #测试点个数
x11 = -0.12; x12 = 0.12;
y11 = -0.12; y12 = 0.12;
z11 = 0.05; z12 = 0.25;

xx1 = np.array([[x11], [y11], [z11], [0], [0]])
xx2 = np.array([[x12], [y12], [z12], [np.pi/2], [np.pi/2]])
Traj = np.empty((200, 5,1))
for i in range(length):
    Traj[i, :] = xx1 + (xx2 -xx1) * np.random.rand(5,1)

In [9]:
# 磁矩M
mag_diameter = 6.05e-3
mag_length = 1.25e-3
mag_BR = 14800
mag_BT = mag_BR* (mag_diameter**2) * (mag_length / 16)
M = np.empty((200,3,1))
for i in range(length):
    M[i, :] = np.array([np.sin(Traj[i, 3]) * np.cos(Traj[i, 4]), np.sin(Traj[i, 3]) * np.sin(Traj[i, 4]), np.cos(Traj[i, 4])])



In [10]:
theoryData = np.empty((16, 200, 3, 1))
for j in range(16):
    for i in range(200):
        R = np.linalg.norm(SensorPosition[j] - Traj[i, 0:3])
        theoryData[j, i] = (mag_BT * 3 * np.dot(M[i].T, SensorPosition[j] - Traj[i, 0:3])* (SensorPosition[j] - Traj[i, 0:3])- (R **2) * M[i] )/ (R ** 5)


In [11]:
# 16个传感器增益
Gain = np.empty((16, 3, 1),dtype=np.int)
for i in range(16):
    Gain[i] = np.array([[1800], [1800], [1800]]) +  200 * np.random.rand(3,1)

#传感器偏置
Offset = np.empty((16, 3, 1))
for i in range(16):
    Offset[i] = np.random.uniform(-0.05, 0.05, (3,1))

#传感器旋转矩阵
H = np.ones((16,3,1))

<ipython-input-11-99f749967be9>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Gain = np.empty((16, 3, 1),dtype=np.int)


In [12]:
import test
params = np.zeros((9,1))
params[0, 0] = Gain[0,0,0]
params[1, 0] = Gain[0,1,0]
params[2, 0] = Gain[0,2,0]
params[3, 0] = H[0,0,0]
params[4, 0] = H[0,1,0]
params[5, 0] = H[0,2,0]
params[6, 0] = Offset[0,0,0]
params[7, 0] = Offset[0,1,0]
params[8, 0] = Offset[0,2,0]

data_input = theoryData[0]


In [13]:
params = np.zeros((9, 1))
params[0, 0] = 1600
params[1, 0] = 1600
params[2, 0] = 1600
params[3, 0] = 0.93
params[4, 0] = 0.94
params[5, 0] = 0.95
params[6, 0] = 0.01
params[7, 0] = -0.005
params[8, 0] = 0.0006

G1 = params[0, 0]
G2 = params[1, 0]
G3 = params[2, 0]
H1 = params[3, 0]
H2 = params[4, 0]
H3 = params[5, 0]
O1 = params[6, 0]
O2 = params[7, 0]
O3 = params[8, 0]


In [29]:
R = test.EulerToMatrix(np.array([[H1], [H2], [H3]]))

In [30]:
np.diag([G1,G2,G3]).dot(R)

array([[1599.56477773,  -26.09831653,   26.67207772],
       [  26.52421907, 1599.57639761,  -25.5306487 ],
       [ -26.2485744 ,   25.96586403, 1599.57393897]])

In [31]:
np.dot(np.diag([G1,G2,G3]), R)

array([[1599.56477773,  -26.09831653,   26.67207772],
       [  26.52421907, 1599.57639761,  -25.5306487 ],
       [ -26.2485744 ,   25.96586403, 1599.57393897]])

In [32]:
test.my_Func(params,theoryData[0])

IndexError: index 1 is out of bounds for axis 0 with size 1